# 🤖 Entraînement RL Agent sur Google Colab

Ce notebook est **spécifiquement pour entraîner l'agent RL** sur Colab avec GPU.
Le modèle entraîné sera téléchargé pour utilisation locale.


## 📦 Étape 1 : Installation des dépendances


In [1]:
# Installation PyTorch avec CUDA 11.8 (pour GPU Colab)
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 -q

# Dépendances principales pour RL
%pip install diffusers transformers accelerate -q
%pip install stable-baselines3[extra] gymnasium -q
%pip install pillow numpy requests pydantic pydantic-settings -q

print("✅ Dépendances installées")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 6.9 MB/s eta 0:00:00
✅ Dépendances installées


## 🔧 Étape 2 : Cloner et configurer le projet


In [ ]:
# ⚠️ MODIFIER CETTE URL avec votre repository GitHub
REPO_URL = "https://github.com/Hasso2021/Projet_fil_rouge_ML_D.git"

import os

# Cloner le repo
# Note: Le nom du dossier cloné dépend du nom du repository GitHub
# Le dossier créé sera automatiquement le nom du repository (sans .git)
repo_name = REPO_URL.split("/")[-1].replace(".git", "")

print(f"📥 Clonage du repository GitHub: {REPO_URL}")

if not os.path.exists(repo_name):
    # Cloner le repository
    result = os.system(f"git clone {REPO_URL}")
    if result != 0:
        print(f"❌ Erreur lors du clonage du repository")
        print(f"💡 Vérifiez que:")
        print(f"   1. L'URL du repository est correcte: {REPO_URL}")
        print(f"   2. Le repository est public (ou vous êtes authentifié)")
        print(f"   3. Votre connexion internet fonctionne")
        raise Exception("Échec du clonage du repository")
    print(f"✅ Repository cloné avec succès")
else:
    print(f"✅ Repository déjà présent: {repo_name}")

# Aller dans le dossier cloné
os.chdir(repo_name)
print(f"✅ Dossier actuel: {os.getcwd()}")

# Configuration pour Colab (GPU) - Entraînement RL
os.environ["SD_DEVICE"] = "cuda"
os.environ["SD_DTYPE"] = "float16"
os.environ["OUTPUT_DIR"] = "outputs"
os.environ["MODELS_DIR"] = "models"
os.environ["RL_AGENT_PATH"] = "models/rl_agent.zip"

print("✅ Projet configuré pour entraînement RL")


✅ Dossier actuel: /content/Projet_fil_rouge_ML_D/Projet_fil_rouge_ML_D
✅ Projet configuré pour entraînement RL


## 🔍 Étape 3 : Vérifier le GPU


In [4]:
!nvidia-smi

import torch
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("\n⚠️ IMPORTANT: Assurez-vous d'avoir un GPU activé pour l'entraînement RL")
else:
    print("\n❌ ERREUR: Aucun GPU détecté. Activez le GPU dans Runtime > Change runtime type")


Thu Nov 20 08:32:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 🚀 Étape 4 : Entraîner l'agent RL

**⚠️ IMPORTANT :**
- Entraînement 2.5k steps : ~30 minutes sur GPU (configuration actuelle - rapide pour tests)
- Entraînement 10k steps : ~1-2 heures sur GPU (recommandé pour meilleure qualité)
- Entraînement 20k steps : ~2-4 heures sur GPU (optimal)
- Le modèle sera sauvegardé dans `models/rl_agent.zip`


In [ ]:
# Configuration de l'entraînement
# Réduit à 2500 steps pour entraînement rapide (~30 minutes)
# Pour meilleure qualité : augmenter à 10000 steps (~1-2 heures)
TOTAL_TIMESTEPS = 2500  # Configuration rapide pour tests (30 minutes)
SAVE_PATH = "models/rl_agent.zip"

print(f"🚀 Démarrage entraînement RL agent...")
print(f"📊 Steps d'entraînement: {TOTAL_TIMESTEPS}")
print(f"⏱️ Temps estimé: {TOTAL_TIMESTEPS // 5000 * 60:.0f}-{TOTAL_TIMESTEPS // 2500 * 30:.0f} minutes")
print(f"💡 Note: Modèle rapide pour tests. Pour meilleure qualité, utiliser 10000 steps (1-2h)")
print("\n" + "="*50 + "\n")

# Importer les modules
from app.models.rl_agent import RLOptimizer
from training.rl_env import PromptOptimizationEnv

# Créer environnement et agent
env = PromptOptimizationEnv()
agent = RLOptimizer(env=env)

# Entraîner l'agent
agent.train(
    total_timesteps=TOTAL_TIMESTEPS,
    save_path=SAVE_PATH
)

print("\n" + "="*50)
print("✅ Entraînement terminé !")
print(f"📦 Modèle sauvegardé: {SAVE_PATH}")


🚀 Démarrage entraînement RL agent...
📊 Steps d'entraînement: 2500
⏱️ Temps estimé: 0-30 minutes
💡 Note: Modèle rapide pour tests. Pour meilleure qualité, utiliser 10000 steps (1-2h)




Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.w

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

## ✅ Étape 5 : Vérifier le modèle entraîné


In [ ]:
# Vérifier que le modèle existe et a la bonne taille
import os
model_path = "models/rl_agent.zip"

if os.path.exists(model_path):
    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"✅ Modèle trouvé: {model_path}")
    print(f"📦 Taille: {size_mb:.2f} MB")

    # Lister tous les fichiers du modèle
    import zipfile
    with zipfile.ZipFile(model_path, 'r') as zip_ref:
        files = zip_ref.namelist()
        print(f"📁 Fichiers dans le modèle: {len(files)}")
        for f in files[:5]:  # Afficher les 5 premiers
            print(f"   - {f}")
        if len(files) > 5:
            print(f"   ... et {len(files) - 5} autres fichiers")
else:
    print(f"❌ ERREUR: Modèle non trouvé à {model_path}")


## 💾 Étape 6 : Sauvegarder dans Google Drive (Optionnel)

**⚠️ Note** : Cette étape est **optionnelle**. Si vous avez des erreurs avec Google Drive, passez directement à l'Étape 7 (téléchargement direct).

**Erreur "credential propagation was unsuccessful" ?**
- Solution 1 : Réessayer après quelques secondes
- Solution 2 : Utiliser l'Étape 7 (téléchargement direct) à la place
- Solution 3 : Rafraîchir la page Colab et réessayer


In [ ]:
# Monter Google Drive pour sauvegarder le modèle (OPTIONNEL)
# Si cette cellule échoue, passez à l'Étape 7 pour téléchargement direct

try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Créer dossier de sauvegarde
    DRIVE_MODELS_DIR = "/content/drive/MyDrive/ai-creative-studio/models"
    os.makedirs(DRIVE_MODELS_DIR, exist_ok=True)
    
    # Copier le modèle vers Drive
    model_name = "rl_agent.zip"
    drive_model_path = f"{DRIVE_MODELS_DIR}/{model_name}"
    
    if os.path.exists("models/rl_agent.zip"):
        os.system(f"cp models/rl_agent.zip {drive_model_path}")
        print(f"✅ Modèle sauvegardé dans Google Drive")
        print(f"📦 Chemin: {drive_model_path}")
        print(f"\n💡 Vous pourrez télécharger depuis Drive ou utiliser sur Colab")
    else:
        print("⚠️ Modèle non trouvé: models/rl_agent.zip")
        print("💡 Assurez-vous que l'entraînement est terminé avant de sauvegarder dans Drive")
        
except Exception as e:
    print(f"⚠️ Erreur lors du montage de Google Drive: {e}")
    print("\n💡 Solutions:")
    print("   1. Réessayer après quelques secondes")
    print("   2. Rafraîchir la page Colab (F5) et réessayer")
    print("   3. Utiliser l'Étape 7 (téléchargement direct) à la place")
    print("   4. Passer cette étape - le modèle sera téléchargé dans l'Étape 7")


## 📥 Étape 7 : Télécharger le modèle localement

**Option 1 : Téléchargement direct depuis Colab**


In [ ]:
# Télécharger le modèle directement depuis Colab
from google.colab import files

model_path = "models/rl_agent.zip"
if os.path.exists(model_path):
    print(f"📥 Téléchargement de {model_path}...")
    files.download(model_path)
    print("✅ Téléchargement terminé !")
    print("\n💡 Placez le fichier dans le dossier 'models/' de votre projet local")
else:
    print(f"❌ Modèle non trouvé: {model_path}")


**Option 2 : Télécharger depuis Google Drive**

1. Allez sur [Google Drive](https://drive.google.com)
2. Naviguez vers `ai-creative-studio/models/`
3. Téléchargez `rl_agent.zip`
4. Placez-le dans `models/rl_agent.zip` de votre projet local


## 🧪 Étape 8 : Test rapide du modèle (optionnel)

Tester que le modèle fonctionne correctement avant téléchargement.


In [ ]:
# Test rapide d'optimisation de prompt
test_prompt = "a cat"

print(f"🧪 Test d'optimisation pour: '{test_prompt}'...")

try:
    result = agent.optimize_prompt(
        base_prompt=test_prompt,
        n_iterations=5  # Test rapide avec 5 itérations
    )

    print("\n📊 Résultats:")
    print(f"  Prompt original: {result['original_prompt']}")
    print(f"  Prompt optimisé: {result['optimized_prompt']}")
    print(f"  Score original: {result['original_score']:.2f}")
    print(f"  Score optimisé: {result['optimized_score']:.2f}")
    print(f"  Amélioration: {result['improvement']:+.2f}")
    print(f"  Paramètres: {result['best_params']}")
    print("\n✅ Modèle fonctionne correctement !")
except Exception as e:
    print(f"⚠️ Erreur lors du test: {e}")
    print("💡 Le modèle peut quand même fonctionner localement")
